# Basecalling Bee ONT Data from R10 flowcell

In [6]:
WKDIR=/workspace/hraijc/Bee/BeeT/BeeT_ONT14_1_basecalling
READDIR=/input/genomic/insect/Leioproctus/imitatus/ExperimentRequestor10992/BeeT_ONT14_1/20230608_1019_MN19482_FAW89221_7fa32855/pod5/
TESTDIR=/workspace/hraijc/TEMP/Bee_Test
DORADO=/workspace/hraijc/git_clones/dorado/dorado-0.3.0-linux-x64/bin/dorado
MODELS=/workspace/hraijc/git_clones/dorado/dorado-0.3.0-linux-x64/models

In [7]:
cd $WKDIR
mkdir -p ${WKDIR}/log

In [5]:
#Get printout of workflows
# Using non-GPU guppy

sbatch << EOF
#!/bin/bash
#SBATCH -J Flye
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=1:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err

module load guppy/6.4.6 
guppy_basecaller --help
module unload guppy


EOF


Submitted batch job 1387939


In [ ]:
# WANT: dna_r10.4.1_e8.2_260bps_hac
# FLO-MIN114     SQK-LSK114
# guppy_basecaller -i <input path> -s <save path> -c <config file>

### Test run on a small file

In [11]:
cp ${READDIR}/FAW89221_7fa32855_2be29263_99* $TESTDIR

In [11]:
#mkdir ${WKDIR}/test1
ls ${TESTDIR}

FAW89221_7fa32855_2be29263_990.pod5  FAW89221_7fa32855_2be29263_996.pod5
FAW89221_7fa32855_2be29263_991.pod5  FAW89221_7fa32855_2be29263_997.pod5
FAW89221_7fa32855_2be29263_992.pod5  FAW89221_7fa32855_2be29263_998.pod5
FAW89221_7fa32855_2be29263_993.pod5  FAW89221_7fa32855_2be29263_999.pod5
FAW89221_7fa32855_2be29263_994.pod5  FAW89221_7fa32855_2be29263_99.pod5
FAW89221_7fa32855_2be29263_995.pod5


In [12]:
#Guppy
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=4G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=1:00:00
#SBATCH --gres=gpu:1

module load guppy/6.4.6-gpu

guppy_basecaller -i ${TESTDIR} -s ${WKDIR}/test1 --flowcell FLO-MIN114 --kit SQK-LSK114 --device auto --recursive

EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 1387991


In [21]:
#DORADO duplex
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=8G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=1:00:00
#SBATCH --gres=gpu:1

cd ${WKDIR}/test1

${DORADO} duplex ${MODELS}/dna_r10.4.1_e8.2_400bps_hac@v4.2.0 ${TESTDIR} --device 'cuda:0' --recursive > BeeT_DoradoTest1.bam

EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 1401332


In [25]:
#DORADO simplex
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=8G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=1:00:00
#SBATCH --gres=gpu:1

cd ${WKDIR}/test1

${DORADO} basecaller ${MODELS}/dna_r10.4.1_e8.2_400bps_hac@v4.2.0 ${TESTDIR} --device 'cuda:0' --recursive > BeeT_DoradoTestS2.bam

EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 1401341


In [12]:
du -sh ${WKDIR}/test1/pass/
du -sh ${WKDIR}/test1/fail/


du: cannot access ‘/workspace/hraijc/Bee/BeeT/BeeT_ONT14_1_basecalling/test1/pass/’: No such file or directory
du: cannot access ‘/workspace/hraijc/Bee/BeeT/BeeT_ONT14_1_basecalling/test1/fail/’: No such file or directory


: 1

In [18]:
#Guppy
seff 1387991

Job ID: 1387991
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 4
CPU Utilized: 00:05:26
CPU Efficiency: 44.54% of 00:12:12 core-walltime
Job Wall-clock time: 00:03:03
Memory Utilized: 534.66 MB
Memory Efficiency: 13.05% of 4.00 GB


In [22]:
#DoradoDuplex
seff 1401332

Job ID: 1401332
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 4
CPU Utilized: 00:03:29
CPU Efficiency: 39.29% of 00:08:52 core-walltime
Job Wall-clock time: 00:02:13
Memory Utilized: 3.88 GB
Memory Efficiency: 48.53% of 8.00 GB


In [26]:
#DoradoDuplex
seff 1401341

Job ID: 1401341
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 4
CPU Utilized: 00:02:35
CPU Efficiency: 26.36% of 00:09:48 core-walltime
Job Wall-clock time: 00:02:27
Memory Utilized: 3.02 GB
Memory Efficiency: 37.75% of 8.00 GB


In [27]:
# Get QC report of runs. 

sbatch << EOF
#!/bin/bash
#SBATCH -J NanoPlot
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=1G
#SBATCH --time=01:02:00

source deactivate
module load nanopack/1.1.1


NanoPlot -t 2 --ubam ${WKDIR}/test1/BeeT_DoradoTest1.bam -o ${WKDIR}/test1/summary-plots
NanoPlot -t 2 --ubam ${WKDIR}/test1/BeeT_DoradoTestS2.bam -o ${WKDIR}/test1/summary-plots-s2


EOF

Submitted batch job 1401342


## Dorado conclusions:
- Duplex mode is faster and gives more accurate reads in smaller datasets but takes longer in larger datasets.
- benchmarking info here: https://aws.amazon.com/blogs/hpc/benchmarking-the-oxford-nanopore-technologies-basecallers-on-aws/
- Dorado works best on 1 GPU with 60GB RAM and 8vCPU

## Basecalling

In [7]:
#Guppy
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=6G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=11:00:00
#SBATCH --gres=gpu:1

module load guppy/6.4.6-gpu

guppy_basecaller -i ${READDIR} -s ${WKDIR}/guppy --flowcell FLO-MIN114 --kit SQK-LSK114 --device auto --recursive

EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 1401302


In [32]:
#DORADO duplex
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=61G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=24:00:00
#SBATCH --gres=gpu:1

cd ${WKDIR}/dorado

${DORADO} duplex ${MODELS}/dna_r10.4.1_e8.2_400bps_hac@v4.2.0 ${READDIR} --device 'cuda:0' --recursive > BeeT_DoradoDuplex.bam

EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 1401348


In [31]:
#DORADO simplex
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=6
#SBATCH --mem=8G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=11:00:00
#SBATCH --gres=gpu:1

cd ${WKDIR}/dorado

${DORADO} basecaller ${MODELS}/dna_r10.4.1_e8.2_400bps_hac@v4.2.0 ${READDIR} --device 'cuda:0' --recursive > BeeT_DoradoSimplex.bam

EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 1401345


In [5]:
# Get QC report of runs. 

sbatch << EOF
#!/bin/bash
#SBATCH -J NanoPlot
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem-per-cpu=6G
#SBATCH --time=02:02:00

source deactivate
module load nanopack/1.1.1


NanoPlot -t 12 --summary ${WKDIR}/guppy/sequencing_summary.txt -o ${WKDIR}/guppy/NanoPlot-GS
NanoPlot -t 12 --ubam ${WKDIR}/dorado/BeeT_DoradoDuplex.bam -o ${WKDIR}/dorado/NanoPlot-DD
NanoPlot -t 12 --ubam ${WKDIR}/dorado/BeeT_DoradoSimplex.bam -o ${WKDIR}/dorado/NanoPlot-DS


EOF

Submitted batch job 1402825


In [3]:
seff 1401302
seff 1401345
seff 1401348


Job ID: 1401302
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 6
CPU Utilized: 15:01:55
CPU Efficiency: 32.99% of 1-21:33:36 core-walltime
Job Wall-clock time: 07:35:36
Memory Utilized: 1.46 GB
Memory Efficiency: 24.31% of 6.00 GB
Job ID: 1401345
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 6
CPU Utilized: 06:30:54
CPU Efficiency: 24.26% of 1-02:51:24 core-walltime
Job Wall-clock time: 04:28:34
Memory Utilized: 4.00 GB
Memory Efficiency: 50.02% of 8.00 GB
Job ID: 1401348
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 8
CPU Utilized: 09:50:51
CPU Efficiency: 21.50% of 1-21:47:36 core-walltime
Job Wall-clock time: 05:43:27
Memory Utilized: 8.35 GB
Memory Efficiency: 13.70% of 61.00 GB


## Fastq Generation and file filtering:

In [16]:
# Bam -> fastq samtools
# Filtering using seqkit


sbatch << EOF
#!/bin/bash
#SBATCH -J NanoPlot
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem-per-cpu=6G
#SBATCH --time=02:02:00

module load samtools/1.16

#samtools fastq -@ 12 ${WKDIR}/dorado/BeeT_DoradoDuplex.bam > ${WKDIR}/fastq/BeeT_all.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_all.fastq -j 12 -Q 10 > ${WKDIR}/fastq/BeeT_Q10.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q10.fastq -j 12 -Q 20 > ${WKDIR}/fastq/BeeT_Q20.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q10.fastq -j 12 -m 10000 > ${WKDIR}/fastq/BeeT_Q10L10k.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q10.fastq -j 12 -Q 15 > ${WKDIR}/fastq/BeeT_Q15.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q15.fastq -j 12 -Q 18 > ${WKDIR}/fastq/BeeT_Q18.fastq
/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q10.fastq -j 12 -m 2000 > ${WKDIR}/fastq/BeeT_Q10L2k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q15.fastq -j 12 -m 2000 > ${WKDIR}/fastq/BeeT_Q15L2k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q18.fastq -j 12 -m 2000 > ${WKDIR}/fastq/BeeT_Q18L2k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit seq ${WKDIR}/fastq/BeeT_Q20.fastq -j 12 -m 2000 > ${WKDIR}/fastq/BeeT_Q20L2k.fastq


EOF

Submitted batch job 1402953


In [17]:
# seqstats


sbatch << EOF
#!/bin/bash
#SBATCH -J NanoPlot
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=1G
#SBATCH --time=00:30:00

#/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_all.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q10.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q15.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q18.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q20.fastq
#/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q10L10k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q10L2k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q15L2k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q18L2k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit stat ${WKDIR}/fastq/BeeT_Q20L2k.fastq


EOF




Submitted batch job 1402957
